In [ ]:
import urllib.request, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import warnings

QC = "https://kustom.radio-canada.ca/coronavirus/canada_quebec"

REGIONS = [
    'montreal',
    'monteregie',
    'laval',
    'laurentides',
    'lanaudiere',
    'outaouais',
    'capitale-nationale',
    
    'nunavik',
    'nord-du-quebec',
    'abitibi-temiscamingue',
    'estrie',
    'mauricie-centre-du-quebec',
    'chaudiere-appalaches',
    'bas-saint-laurent',
    'gaspesie-iles-de-la-madeleine',
    'cote-nord',
    'saguenay-lac-saint-jean'
]

In [ ]:
# Data Settings
LAST_DAYS = 600
NTH_LABEL = 8

MA_WINDOW = 4
EXTRAPOLATE_TO = MA_WINDOW * 2

# Plot Settings
# plt.style.use('dark_background')
%matplotlib inline

# Notebook settings
warnings.filterwarnings('ignore')

In [ ]:
def plot(df, Region=None, Hospitalizations=False, ICU=False, Change=False, MA=False, New=False, Deaths=False, Tests=False, Vaccinations=False, Annotate=True, Extrapolate=False):
    fig = plt.figure(figsize = (15,10))
    
    if (Hospitalizations and 'H' in df):
        plt.plot(df['H'], label="Hosp. (incl. ICU)")
        
        if (Extrapolate):
            extrapolate(EXTRAPOLATE_TO, 'H', df, plt)
    
    if (ICU and 'I' in df):
        plt.plot(df['I'], label="ICU")
        
        if (Extrapolate):
            extrapolate(EXTRAPOLATE_TO, 'I', df, plt)
    
    if (Change):
        plt.plot(df['Change'], label="Change")
        
        if (Extrapolate):
            extrapolate(EXTRAPOLATE_TO, 'Change', df, plt)
        
    if (MA):
        plt.plot(df['MA'], label="%s DMA (People)" % MA_WINDOW)
        
        if (Extrapolate):
            extrapolate(EXTRAPOLATE_TO, 'MA', df, plt)
        
    if (New):
        plt.plot(df['New'], label="%s DMA New Positive" % MA_WINDOW, color="fuchsia")
        
        if (Extrapolate):
            extrapolate(EXTRAPOLATE_TO, 'New', df, plt)
        
    if (Deaths):
        plt.plot(df['D'], label="Deaths (Total)", color="black")
        
        if (Extrapolate):
            extrapolate(EXTRAPOLATE_TO, 'D', df, plt)
            
    if (Vaccinations):
        plt.plot(df['V'], label="Vaccinations (Total)", color="blue")
        
        if (Extrapolate):
            extrapolate(EXTRAPOLATE_TO, 'V', df, plt)
    
    if (Tests):
        plt.plot(df['T'], label="Tests Conducted (Millions)", color="green")
    
    plt.title(Region.title())
    plt.xlabel("Days (last %s ending %s)" % (LAST_DAYS, np.datetime_as_string(df.tail(1)['Date'], unit='D')[0]))
    
    if (Tests):
        plt.ylabel("Tests (millions)")
    elif (Vaccinations):
        plt.ylabel("Vaccinations")
    else:
        plt.ylabel("Cases")
    
    # Annual markers
    if (LAST_DAYS > 365):
        ticks = list(range(len(df.index), 0, -365))
        
        if len(ticks) > 0:
            plt.vlines(ticks, ymin=0, ymax=plt.ylim()[1], linestyle='dotted', label="Year")
        
        plt.grid(b=True, which='Major', axis='y')
    else:
        plt.grid(True)
    

    if (Annotate):
        for i, row in df.iterrows():
#             if i % NTH_LABEL == 0 or i == df.index[-1]:
            if i == df.index[-1]:
                if (ICU):
                    plt.annotate(str(row['I']), (i, row['I']))

                if (Hospitalizations):
                    plt.annotate(str(row['H']), (i, row['H']))
                    
                if (MA):
                    plt.annotate(str(row['Active']), (i, row['MA']))
                    plt.annotate(str(row['Change']), (i, row['Change']))
                
                if (New):
                    plt.annotate(str(row['New']), (i, row['New']))
                    
                if (Deaths):
                    plt.annotate(str(row['D']), (i, row['D']))
                
                if (Vaccinations):
                    plt.annotate(str(row['V']), (i, row['V']))
                    
                if (Tests):
                    plt.annotate(str(np.round(row['T']/1000000, decimals=1)), (i, row['T']))

    plt.legend(loc='upper left')
    plt.show()

def get(r=None):
    with urllib.request.urlopen("%s_%s" % (QC, r) if r is not None else QC) as response:
        data = json.load(response)[0]['History']
        df = pd.json_normalize(data)

        # Cast types
        df['Date'] = pd.to_datetime(df['Date'])
        numeric_cols = ['C', 'D', 'H', 'I', 'R', 'T', 'V']
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

        # MA calculations make more sense with "today" at the end
        df = df[::-1]
        df.set_index('Date')

        df['Active'] = df.C - df.R - df.D
        df['Change'] = np.around(df['Active'].diff(), decimals=0)

        df = df.sort_values(by=['Date'], ignore_index=True)

        df['MA'] = df['Active'].rolling(window=MA_WINDOW).mean()
        df['New'] = np.around(df['C'].diff().rolling(window=MA_WINDOW).mean(), decimals=0)

        # Last N days
        df = df.tail(LAST_DAYS)
        
        return df

def extrapolate(count, feature, df, plt):
    # Gross fix for Regions that don't have or report data
    try:
        x = df.index[-count:].tolist()
        y = df[feature][-count:].tolist()
        m, b = np.polyfit(x, y, 1)

        for i in range(count):
            plt.plot((max(x), max(x)+i), (y[-1], i * m + y[-1]), 'r-')
    except:
        pass

# Notes

* The following charts are updated at "interesting intervals" (see age at the top, generally daily).
* More populous regions appear near the top.
* The `x DMA` is the x-day (configurable) Daily Moving Average of *active cases* (new - recoveries - deaths).
* `Change` is for the x-day DMA.
* "Quebec" refers to the province as a whole (which is the only jurisdiction that reports Hospitalisations and ICU admissions).

# Data Window

The data window has been updated to show the evolution of the entire pandemic in order to compare the Second Wave with the First Wave.

In [ ]:
qcdf = get()
plot(df=qcdf, Region="Quebec", Change=False, MA=True, Deaths=True, Hospitalizations=True, ICU=True, Extrapolate=True)
plot(df=qcdf, Region="Quebec", Change=False, MA=False, Deaths=False, Hospitalizations=True, ICU=True, Extrapolate=True)
# plot(df=qcdf, Region="Quebec", Change=True, MA=True, Extrapolate=True)
# plot(df=qcdf, Region="Quebec", New=True, Extrapolate=True)
# plot(df=qcdf, Region="Quebec", Hospitalizations=True, ICU=True, Extrapolate=True)
# plot(df=qcdf, Region="Quebec", Vaccinations=True, Extrapolate=True)
# plot(df=qcdf, Region="Quebec", Deaths=True, Extrapolate=True)
# plot(df=qcdf, Region="Quebec", Tests=True)

for r in REGIONS:
    df = get(r)
    plot(df=df, Region=r, Change=False, MA=True, New=True, Extrapolate=True, Vaccinations=False)